## Programming

### Variables

Create a variable

Display the variable


### Functions

Define a function

Run the function

### Logic

In [ ]:
if():
    print("True")
else:
    print("False")

### Packages

In [ ]:
import numpy as np

In [ ]:
np.array([...])

In [ ]:
np.log()

## Sourcing Data

Import the Quandl API Package

Get the Apple Stock Price Data

In [ ]:
data = quandl.get("", rows=5)

Check the type of the data variable

Look at the data

Get just the closing values of the stocks at the end of each month.

Look at the data

#### Get the data for the last 10 years

Create a start date variable

In [ ]:
import datetime
start_date = (datetime.datetime.now() - datetime.timedelta(days=10*365)).strftime('%Y-%m-01')

Import the Apple data for the last 10 years

In [ ]:
data = quandl.get("WIKI/AAPL.4", rows=120, collapse='monthly', start_date=start_date)
data.head(10)

Visualise the data as a sense check

In [ ]:
%matplotlib inline
!pip install mpld3 # install a package to let us zoom into our plots
import mpld3
mpld3.enable_notebook()

ax = data.plot()

Import the 11th column instead of the 4th

Plot the data again

### Source the full portfolio

In [ ]:
from pandas import read_csv
buffett = read_csv('')
buffett.head(5)

Get the list of Ticker Symbols

In [ ]:
symbols = [ 'WIKI/%s.11' % ticker for ticker in buffett['TICKER'][0:3] ]

Look at the array as a sense check

Pull the data for the three stocks. 

In [ ]:
data2 = quandl.get(symbols, rows=121, collapse='monthly', start_date=start_date)

Have a look at the top of the data

Have a look at the bottom of the data

Clean up the column names and have another look at the data

In [ ]:
data2.columns = [col.replace(' - Adj. Close','').replace('WIKI/','') for col in data2.columns]
data2.tail(5)

Plot the data

### Analyse

Upload the training data

Import the training data into a variable called df

In [ ]:
import pandas as pd
df = pd.read_csv()
df.head(5)

Plot the data

Produce the diff column

In [ ]:
df['diff_hats_sold'] = df.hats_sold - df.hats_sold.shift()
df.head(10)

Plot the diff

In [ ]:
df.plot(y='diff_hats_sold')

Take the logs and plot it

In [ ]:
df['log_hats_sold'] = np.log(df.hats_sold)
df.plot(y='log_hats_sold')

Take the diff of the logs and plot it

In [ ]:
df['difflog_hats_sold'] = df.log_hats_sold - df.log_hats_sold.shift()
df.plot(y='difflog_hats_sold')

Load the model

In [ ]:
from statsmodels.tsa.arima_model import ARIMA

Train the model on our hat data

In [ ]:
model = ARIMA(df.log_hats_sold, order=(2, 1, 2))  
results_ARIMA = model.fit()

Check the model fit visually

In [ ]:
import matplotlib.pyplot as plt
plt.plot(df.difflog_hats_sold)
plt.plot(results_ARIMA.fittedvalues, color='red')

Create our custom functions

In [ ]:
def ARIMAforecast(series, params = [6,1, 2], steps = 20):   
    
    X = series                                                     #take the data
    log_X = np.log(X)                                              #log the data
    model = ARIMA(log_X, order=(params[0], params[1], params[2]))  #create ARIMA model
    results_ARIMA = model.fit(disp =1)                              #fit ARIMA model
    forecast = results_ARIMA.forecast(steps = steps)[0]            #forecast 'steps' number of steps
    unlog_forecast = np.exp(forecast)                              #"unlog" the forecast
    full_predict = np.append(X,unlog_forecast)                     #append it to the end of the column
    return full_predict.tolist()                                   #return as list


def plot_ARIMA(series, test_points, forecast_points, AR, I, MA = 2):
    predictions = ARIMAforecast(series[:-test_points], params = [AR,I,MA], steps = forecast_points)
    _,ax = plt.subplots()
    _,_,_ = ax.plot(predictions, color = "red"), ax.plot(series.values, color = "blue"), plt.show()
    
from ipywidgets import interact, interactive, fixed, interact_manual
def custom_ARIMA(series):    
    interact(plot_ARIMA, series=fixed(series), test_points=(1,10,1), forecast_points=(1,50,1), AR=(1,9,1), I=(1,2,1), MA=(0,4,1))

custom_ARIMA(df.hats_sold)    

Test out model on the apple data

In [ ]:
apple = data.copy()
apple.columns = ['close']
custom_ARIMA(apple.close)

Forecast for all three series

In [ ]:
import pandas as pd

numberofmonths = 6 # how many predictions do we want to make?

# Here we need to add numberofmonths onto the end of the dataframe using data.reindex:
data3 = data2.reindex(pd.date_range(datetime.datetime.now().date(), periods=121 + numberofmonths, freq='MS')+pd.DateOffset(days=-1, months=-120),fill_value="NaN")

data3 = data3.iloc[1:]

master_params = [7,1,0]

for i, column in enumerate(data3):
    if data3[column].isnull().values.any(): # select only the range of dates for which we have data
        last_nan = np.where(data3[column].isnull().values)[0][-1]
        stock = data3[column].iloc[(last_nan+1):-numberofmonths].values
    else:
        stock = data3[column].iloc[:-numberofmonths].values
    
    stock = stock.tolist()
    forecast = ARIMAforecast(stock, params = master_params, steps = numberofmonths) # create our forecast
    
    col_forecast = pd.Series(forecast[-numberofmonths:], index = pd.date_range(start= datetime.datetime.now().date(), periods=numberofmonths, freq='MS') + pd.DateOffset(days=-1, months=1))
    data3[column].iloc[-numberofmonths:] = col_forecast
    

data3.tail(10)

Plot the results

In [ ]:
fig, ax = plt.subplots()
_ = ax.plot(data3.iloc[:-numberofmonths])
_ = ax.plot(data3.iloc[-(numberofmonths+1):], color = "Red")

Export as a CSV in the right format

In [ ]:
datalist=data3.unstack()
csv = datalist.to_csv(header=True, index_label=['symbol','date','price'], date_format='%b %Y', index=True)
csv = csv.replace("price,0","price") # remove addition of ',0' on first line
print csv

import base64
from IPython.display import HTML

b64 = base64.b64encode(csv.encode())
payload = b64.decode()
html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{filename}</a>'
html = html.format(payload=payload,title="stocks.csv",filename="stocks.csv")
HTML(html)